In [1]:
import nltk 
# from nltk.stem.lancaster import LancasterStemmer
# nltk.download('punkt')
import time 
# stemmer = LancasterStemmer()

import numpy 
import tflearn 
import tensorflow 
from tensorflow.python.framework import ops

import random 
import json 
import pickle
from snowballstemmer import TurkishStemmer
turkStem = TurkishStemmer()


with open('intents_.json') as file:
        data = json.load(file)


try: 
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except: 
    
    words = []
    labels = []
    docs_x = []
    docs_y = [] #part 2

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            
            words.extend(wrds) #iki diziyi birleştirir.
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            
        if intent["tag"] not in labels:
            labels.append(intent["tag"])     
            
    words = [turkStem.stemWord(w.lower()) for w in words if w != "?"] #pattern içerisindeki kelimelerin parçalanması
    words = sorted(list(set(words)))
    
    labels = sorted(labels)
    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]


# until part 2
for x, doc in enumerate(docs_x):
    bag = []

    wrds = [turkStem.stemWord(w.lower()) for w in doc]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)
    

training = numpy.array(training)
output = numpy.array(output)

# with open("data.pickle","wb") as f:
#     pickle.dump((words, labels, training, output), f)

# 3-part
ops.reset_default_graph()
# tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)


# model.load("model.tflearn")
#batch_size : girdilerin sayısı
model.fit(training, output, n_epoch=1000, batch_size=90, show_metric=True)
model.save("model.tflearn")

Training Step: 3999  | total loss: 3.20751 | time: 0.013s
| Adam | epoch: 1000 | loss: 3.20751 - acc: 0.5007 -- iter: 270/356
Training Step: 4000  | total loss: 3.04825 | time: 0.017s
| Adam | epoch: 1000 | loss: 3.04825 - acc: 0.5180 -- iter: 356/356
--
INFO:tensorflow:C:\Users\asus\Documents\Python Gazi Chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [turkStem.stemWord(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

def chat():
    
    print("Merhaba benim adım Gazi:) \nÜniversitemiz hakkında merak ettiklerini bana sorabilirsin.")
    while True:
        inp = input("Siz: ")
        if inp.lower() == "quit":
            break
        results = model.predict([bag_of_words(inp, words)])
        
        results_index = numpy.argmax(results)
        print('********************\n')
        tag = labels[results_index]
        print("TAG == ",tag)
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
        print("Bunu mu aramıştınız: ", tag)
        print()
        inp2 = input("Evet --> 1, Hayır --> 2 : ")
        if inp2 == "1":
            print(responses)
        else :
            print("Doğru sözcüklerle tekrar deneyiniz.")

chat()

Merhaba benim adım Gazi:) 
Üniversitemiz hakkında merak ettiklerini bana sorabilirsin.
Siz: erasmus
********************

TAG ==  erasmus deÄŸiÅŸim programÄ±
Bunu mu aramıştınız:  erasmus deÄŸiÅŸim programÄ±

Evet --> 1, Hayır --> 2 : 1
http://erasmus.gazi.edu.tr/ 
